In [52]:
import requests
import json
import os

from openai import OpenAI

In [53]:
client = OpenAI()

In [54]:
question = "Can I cast Hunter's Mark on an Invisible enemy?"

In [55]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "developer",
            "content": [{
                "type": "text",
                "text": """
                    Users are playing an RPG. They will ask questions pertaining to the rules, setting, lore,
                    and current situation. You will accept the query and return a list of relevant keywords that
                    can be used to locate relevant sections of the source material.

                    You will NOT attempt to answer the questions yourself. Your only role is to extract useful keywords
                    to be used in a search. Organize them in order of likely relevance, from most relevant to least.

                    Return the keywords in a comma delimited format with no quote characters or extraneous punctuation.
                """
            }],
        },
        {
            "role": "user",
            "content": [{
                "type": "text",
                "text": question
            }]
        }
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="Hunter's Mark, Invisible enemy, spellcasting rules, conditions, targeting, combat rules", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [56]:
completion.choices[0].message.content

"Hunter's Mark, Invisible enemy, spellcasting rules, conditions, targeting, combat rules"

In [57]:
def elastic_request(data=None, method=None, url=None):
    if method == None:
        method = requests.get
    if data:
        data = json.dumps(data)    

    return method(f"https://192.168.1.153:9200/{url}",
                  headers={
                    "Content-Type": "application/json",
                    "Accept": "application/json",
                    "Authorization": f"ApiKey {os.getenv('ELASTIC_API_KEY')}"
                  },
                  verify=False,
                  data=data)

In [58]:
response = elastic_request(url="players-handbook-embedded/_search",
                           data={
                                "query": {
                                    "match": {
                                        "attachment.content": {
                                            "query": completion.choices[0].message.content,
                                            "operator": "or",
                                            "boost": 0.7
                                        }
                                    }
                                },
                                "knn": {
                                    "field": "content-embedding",
                                    "k": 10,
                                    "boost": 0.3,
                                    "num_candidates": 10,
                                    "query_vector_builder": {
                                        "text_embedding": { 
                                            "model_id": "open-ai-embeddings", 
                                            "model_text": question 
                                        }
                                    }
                                }
                           })

/home/vscode/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.153'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [59]:
hits = response.json()["hits"]["hits"]

In [60]:
[h["_source"]["attachment"]["content"] for h in hits]

["MONK RANGER\n\nPaladin\n\nClad in plate armor that gleams in the sunlight despite the dust and grime of long travel, a human lays down her sword and shield and places her hands on\n\na mortally wounded man. Divine radiance shines from her hands, the man’s wounds knit closed, and his eyes open wide with amazement.\n\nA dwarf crouches behind an outcrop, his black cloak making him nearly invisible in the night, and watches an orc war band celebrating its recent victory.\n\nSilently, he stalks into their midst and whispers an oath, and two orcs are dead before they even realize he is there.\n\nSilver hair shining in a shaft of light that seems to illuminate only him, an elf laughs with exultation. His spear flashes like his eyes as he jabs again and\n\nagain at a twisted giant, until at last his light overcomes its hideous darkness.\n\nWhatever their origin and their mission, paladins are united by their oaths to stand against the forces of evil. Whether sworn before a god’s altar and th

In [71]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "developer",
            "content": [{
                "type": "text",
                "text": """
                    Users are playing an RPG. They will ask questions pertaining to the rules, setting, lore,
                    and current situation. You will answer the question based on the content provided.
                    
                    Use only the content provided. IMPORTANT: Do NOT base your answers on ANYTHING other than the content itself.
                    Quote the content verbatim if possible (but only if it is relevant to the question!).
                    Otherwise, keep your answer as brief as possible. If the content is not sufficient to answer the question,
                    say so.
                """
                # Note: this is illiberal. it says "cannot answer the question because content says nothing". What we often want is more
                # of the form: "content doesn't say you can't do it, so you can"

                # Also, perhaps a "Maybe?" response is in order? for example, we get this response from "Can I cast Hunter's Mark on an entity that is Invisible?"
                # "Yes, you can cast Hunter\'s Mark on an invisible enemy as long as you can see the creature. The spell states, "You choose a creature you can see within range," and since Hunter\'s Mark does not specify that the target must be visible, you can apply the mark to an invisible creature that you can see"
                # There is some sense to this adjudication of "yes you can, if you can still see them" (e.g. if the invisible entity has had Fairy Fire cast upon it, or if the caster has Blindsight/Truesight)
                #   but it's not exactly a clear adjudication
            }],
        },
        {
            "role": "assistant",
            "content": [{ "type": "text", "text": h["_source"]["attachment"]["content"]} for h in hits[0:3]]
        },
        {
            "role": "user",
            "content": [{
                "type": "text",
                "text": question
            }]
        }
    ]
)

In [72]:
completion.choices[0].message.content

"Yes, you can cast Hunter's Mark on an invisible enemy. The spell allows you to choose a creature you can see within range, and while the enemy is invisible, you can still target it. However, you must be able to determine its location or have some means of locating it. "